# Código Base

## Librerias

In [16]:
import os
import sys
import comtypes.client
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Conexión con ETABS ya iniciado
* False =  Se abre el programa
* True = Trabaja con el programa ya iniciado

In [17]:
AttachToInstance = False

## Arrancar ETABS y dirección de "SAVE"

* Se especifica de dónde se abre
* Se especifica donde se guardará por defecto en caso de no haberlo abierto previamente
* Se especifica nombre del archivo en caso se cree un archivo .edb

In [18]:

SpecifyPath = True

# Especificar la ruta de ETABS a continuación
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 20\ETABS.exe"



# crear objeto API helper
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

if AttachToInstance:
    try:
        # obtener el objeto ETABS activo
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            # crear una instancia del objeto ETABS a partir de la ruta especificada
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            # crear una instancia del objeto ETABS a partir del último ETABS instalado
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    # Comenzar la aplicacion de ETABS
    myETABSObject.ApplicationStart()

# Crear un objeto SapModel
ret = SapModel = myETABSObject.SapModel

# Iniciar Modelo
ret = SapModel.InitializeNewModel()

ret

0

### Abrir un ETABS existente

In [13]:
#FileName="C:\PRUEBACURSO_PYTHON_ETABS\NOVER.edb"
FileName = "C:\\PRUEBACURSO_PYTHON_ETABS\\NOVER.edb"
ret = SapModel.File.OpenFile(FileName)
ret

0

## Pasos básicos para iniciar ETABS

### Crear grillas
* Se crean número de pisos, grillas y espaciamientos
* Solo se puede utilizar esto en pies
* Solo se puede utilizar antes del newblank

In [12]:
# Crear Grillas
ret = SapModel.File.NewGridOnly(10,   # Número de pisos
                                3*3.28084,   # altura típico de piso a partir del segundo
                                4*3.28084,   # Altura del primer nivel
                                7,    # Número de grillas en X
                                7,     # Número de grillas en Y
                                3.5*3.28084,    # Espaciamiento en X
                                4*3.28084)    # Espaciamiento en Y

### Grilla de pórticos automático

In [10]:
ret = SapModel.File.NewSteelDeck(4,12,12,4,4,24,24)

### Crear modelo en blanco
* El uso de grillas funciona como un NewBlank

In [17]:
# Crear un modelo en blanco
ret =  SapModel.File.NewBlank()

### Colocar unidades

In [14]:
# Ton_m_C = 12
tonf_m_C = 12
ret = SapModel.SetPresentUnits(tonf_m_C)

## Establecer número de pisos y alturas

In [15]:
#ret = SapModel.Story.SetElevation("Story1", 3)

#ret = SapModel.Story.SetMasterStory("Story1", True)


inStoryNames = ["Story1", "MyStory2", "MyStory3", "MyStory4", "MyStory5", "MyStory6"]
inStoryHeights = [3, 3, 3, 3, 3, 3]
inIsMasterStory = [True, False, False, False, False, False]
inSimilarToStory =      ["None", "Story1", "Story1", "Story1", "Story1", "Story1"]
inSpliceAbove = [False, False, False, False, False, False]
inSpliceHeight = [0,0,0,0,0,0]
inColor = [65535, 0, 255, 16711935, 16711680, 0]        

ret = SapModel.Story.SetStories_2(0, 6, inStoryNames, inStoryHeights, inIsMasterStory, 
                                           inSimilarToStory, inSpliceAbove, inSpliceHeight, inColor)

ret


[('Story1', 'MyStory2', 'MyStory3', 'MyStory4', 'MyStory5', 'MyStory6'),
 (3.0, 3.0, 3.0, 3.0, 3.0, 3.0),
 (True, False, False, False, False, False),
 ('None', 'Story1', 'Story1', 'Story1', 'Story1', 'Story1'),
 (False, False, False, False, False, False),
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 (65535, 0, 255, 16711935, 16711680, 0),
 1]

## Creación de materiales

### Creación del concreto


- Concreto Armado   = 2
- Acero de Refuerzo = 6
- Albañileria       = 8

In [17]:
# Definir Propiedades del Material
MATERIAL_CONCRETE = 2
fc=2100 # Resistencia del concreto

ret = SapModel.PropMaterial.SetMaterial('CONC 210',          # Name
                                   MATERIAL_CONCRETE            # Material Type
                                        )


# Definición de las propiedades no lineales
ret = SapModel.PropMaterial.SetOConcrete_1('CONC 210', # Nombre del concreto
                                           fc,         # resistencia del concreto
                                           False,      # True es sí a ser concreto ligero
                                           0,          # FcsFactor (Double)
                                           2,          # 0 user defined, 1 tipo simple, 2 Mander
                                           2, # 2 Takeda, 3 pivot, 4 concrete, 5 BRB, 6 Degrading, 7 Isotropic, 1 Kinematic, 0 elastic
                                           0.002,  # Deformación del concreto en el pico máximo
                                           0.0033,  # Deformación última
                                           -0.1)  # Pendiente de caída 



# Asignar propiedades mecanicas isotropicas al material en MPa
ret = SapModel.PropMaterial.SetMPIsotropic('CONC 210',            # Name
                                    15000*math.sqrt(fc/10)*10,               # E
                                    0.2,                # v
                                    0.0000055           # T
                                    )

# cambiar el peso especifico del material ( 1 es peso)
ret = SapModel.PropMaterial.SetWeightAndMass('CONC 210', 1, 2.4)


## Creación del acero de refuerzo

In [18]:
# Definir Propiedades del Material
MATERIAL_REBAR = 6
fy=42000
fu=63000
fye=fy*1.25
fue=fu*1.25
ret = SapModel.PropMaterial.SetMaterial('ACERO DE REFUERZO', 
                                   MATERIAL_REBAR
                                 )

# Asignar propiedades mecanicas isotropicas al material
ret = SapModel.PropMaterial.SetMPIsotropic('ACERO DE REFUERZO', 
                                    20000000, 
                                    0.3, 
                                    0.0000055
                                    )

 # assign other properties
ret = SapModel.PropMaterial.SetORebar("ACERO DE REFUERZO", fy, fu, fye, fue,
                                       1, # 0 user defined, simple, 2 park
                                       1, # 1 kinematic, 2 Takeda, 3 pivot, 4 concrete, 5 BRB, 6 Degrading, 7 Isotropic, 0 elastic 
                                       0.01, # Deformación de inicio de endurecimiento
                                       0.08, # Deformación final
                                       False) # True = usa por default caltrans para control las deformaciones 

## Crear LoadPattern

- Dead          = 1
- SuperDead     = 2
- Live          = 3
- ReduceLive    = 4
- Quake         = 5
- Wind          = 6
- Snow          = 7
- Other         = 8

In [26]:
# Creación de carga viva
ret = SapModel.LoadPatterns.Add('LP1',    # Name del Load Pattern
                                5,      # Tipo de Load Pattern
                                0,      # Multiplicador de peso propio (0) Solo peso propio es 1
                                True    # Opcional - True: Crear un nuevo caso de carga
                                )

ret

1

## Definir masa sísmica

In [25]:
   #set mass source
# 
ret = SapModel.PropMaterial.SetMassSource(2,                        # 2 es carga especificada, otros 1 y 3
                                          2,                        # 2 es el número de loadpattern
                                          ["DEAD", "LIVE"],         # Cargas de masa sísmica
                                          [1, 0.25])                # % De carga para cada caso anteriormente señalado

ret

[('DEAD', 'LIVE'), (1.0, 0.25), 0]

## Números de modos

In [28]:
ret = SapModel.LoadCases.ModalEigen.SetNumberModes("Modal"         # Nombre del caso modal
                                                   , 18             # Número mínimo de modos
                                                   , 18)            # Número máximo de modos

ret

0

## Definición de vigas y columnas

### Definición de vigas

In [31]:
# Creación de elemento tipo Frame            
# Definir las propiedades de Seccion a un Frame rectangular   
ret = SapModel.PropFrame.SetRectangle('V1',       # Name del nuevo frame
                                'CONC 210',     # Material
                                0.5,         # h
                                0.25          # b
                                )

ret

0

In [34]:
# Acero de refuerzo en vigas

ret = SapModel.PropFrame.SetRebarBeam("V1",       # Name de un frame existente
                                "ACERO DE REFUERZO",    # Material del acero longitudinal
                                "ACERO DE REFUERZO",    # Material del acero transversal
                                0.06,        # Distancia al centro del acero de refuerzo arriba  6cm
                                0.06,          # Distancia al centro del acero de refuerzo abajo  6cm
                                0.000258,        # Top Left Area
                                0.000258,        # Top Right Area
                                0.000258,        # Bottom Left Area
                                0.000258         # Bottom Right Area
                                )

ret

0

### Definición de columnas

In [32]:
# Creación de elemento tipo Frame
# Definir las propiedades de Seccion a un Frame rectangular   
ret = SapModel.PropFrame.SetRectangle('C1',       # Name del nuevo frame
                                'CONC 210',     # Material
                                0.6,         # b
                                0.6          # h
                                )

In [36]:
# Datos extra para Frame Columnas
ret = SapModel.PropFrame.SetRebarColumn("C1",     # Name de un frame existente
                                "ACERO DE REFUERZO",    # Material del acero longitudinal
                                "ACERO DE REFUERZO",    # Material del acero transversal
                                1,          # Pattern  (Rectangular = 1  /   Circular = 2)
                                1,          # Tipo de Cofinamiento  (Ties = 1  /   Spiral = 2)
                                0.04,       # Recubrimiento de la columna  4cm
                                10,         # Solo Aplica para la Columnas Circulares, Numero total de barras
                                6,          # Solo Aplica para la Columnas Rectangulares, Numero de barras en el eje local 3
                                6,          # Solo Aplica para la Columnas Rectangulares, Numero de barras en el eje local 2
                                "#5",       # El diametro del acero longitudinal
                                "#5",       # El diametro del acero transversal
                                0.1,        # Espaciamiento de los estribos
                                2,          # Numero de estribos alrededor del eje local 2
                                2,          # Numero de estribos alrededor del eje local 3
                                False)      # True es diseñado y False es revisado

ret

0

### Definir modificadores de propiedad a un "frame"

In [37]:
# Definir modificadores de propiedad a un Frame
ModValue = [2, 1, 1, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('V1', ModValue)


# Definir modificadores de propiedad a un Frame
ModValue = [2, 1, 1, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('C1', ModValue)

## Definir losa y muro

Tipos de Losa: 
- Slab       = 0
- Drop       = 1
- Ribbed     = 3
- Waffle     = 4
- Mat        = 5
- Footing    = 6

Tipo de elemento Área:
- ShellThin  = 1
- Shellthick = 2
- Menbrane   = 3
- Layered    = 6

Tipo de propiedad (siempre es 1)
- Specified      = 1
- AutoSelectList = 2

### Definición de losa

In [38]:
# Definir las propiedades de la Losa Slab
ret = SapModel.PropArea.SetSlab("Losa Aligerada",       # Name
                        0,                  # Slab Type - Slab
                        3,                  # Membrana
                        "CONC 210",    # Material
                        0.0001                  # Thickness
                        )

### Definición de muro

In [39]:
# Definir propiedades de elementos Muro
ret = SapModel.PropArea.SetWall("Muro15cm",      # Name
                        1,              # Tipo de propiedad
                        1,              # Tipo de elemento área   
                        "CONC 210",         # Material
                        0.15              # Espesor
                        )

### Modificador de propiedad en Muro y Losa

* Value(0) Membrane f11 modifier
* Value(1) Membrane f22 modifier
* Value(2) Membrane f12 modifier
* Value(3) Bending m11 modifier
* Value(4) Bending m22 modifier
* Value(5) Bending m12 modifier
* Value(6) Shear v13 modifier
* Value(7) Shear v23 modifier
* Value(8) Mass modifier
* Value(9) Weight modifier

In [43]:
muroprop = [2, 2, 2, 1, 1, 1, 1, 2, 2,1]
ret = SapModel.PropArea.SetModifiers("Losa Aligerada",
                                      muroprop)

ret

[(2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0), 0]

## Creación de elementos ( Viga, Columna y Losa)

### Creación de viga y columna

In [44]:
# Añadir objeto Frame por Coordenadas
FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
FrameName4 = ' '
FrameName5 = ' '
FrameName6 = ' '
FrameName7 = ' '
FrameName8 = ' '


[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 3, FrameName1, 'C1', '1', 'Global')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 3, 3, 0, 3, FrameName2, 'V1', '2', 'Global')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(3, 0, 0, 3, 0, 3, FrameName3, 'C1', '3', 'Global')
[FrameName4, ret] = SapModel.FrameObj.AddByCoord(0, -3, 0, 0, -3, 3, FrameName4, 'C1', '4', 'Global')
[FrameName5, ret] = SapModel.FrameObj.AddByCoord(0, -3, 3, 3, -3, 3, FrameName5, 'V1', '5', 'Global')
[FrameName6, ret] = SapModel.FrameObj.AddByCoord(3, -3, 0, 3, -3, 3, FrameName6, 'C1', '6', 'Global')
[FrameName7, ret] = SapModel.FrameObj.AddByCoord(0, -3, 3, 0, 0, 3, FrameName7, 'V1', '7', 'Global')
[FrameName8, ret] = SapModel.FrameObj.AddByCoord(3, -3, 3, 3, 0, 3, FrameName8, 'V1', '8', 'Global')

### Creación de elemento losa y Muro

### Creación de losa

In [45]:
x=[0, 0, 3, 3]
y=[0, -3, -3, 0]
z=[3, 3, 3, 3]

ret = SapModel.AreaObj.AddByCoord( 4,         # Número de puntos utilizados en x,y,z
                                   x,         # Puntos coordenados en la dirección X
                                   y,         # Puntos coordenados en la dirección Y
                                   z,         # Puntos coordenados en la dirección Z
                                   "F2",         # Nombre de la losa (no funciona)
                                   "Losa Aligerada")         # Nombre del tipo de sección

### Creación de muro de CA

In [46]:
x=[0, 3, 3, 0]
y=[0, 0, 0, 0]
z=[0, 0, 3, 3]

ret = SapModel.AreaObj.AddByCoord( 4,         # Número de puntos utilizados en x,y,z
                                   x,         # Puntos coordenados en la dirección X
                                   y,         # Puntos coordenados en la dirección Y
                                   z,         # Puntos coordenados en la dirección Z
                                   "F2",         # Nombre de la losa (no funciona)
                                   "Muro15cm")         # Nombre del tipo de sección

## Resfrescar vista

In [56]:
ret = SapModel.View.RefreshView(0, False)

## Asignar Restricciones

### Definir restricción de un punto

In [53]:
# Asignar resitricciones al punto de la Base
PointName1 = ' '
PointName2 = ' '

Restraint = [True, True, True, True, True, True]

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName4, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName6, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)



In [51]:
Restraint = [False, False, False, False, False, False]
ret = SapModel.PointObj.SetRestraint("5", Restraint)
ret

[(False, False, False, False, False, False), 0]

### Definir restricción de toda la base

## Asignación de brazos rígidos

In [59]:
ret = SapModel.SelectObj.All()
ret = SapModel.FrameObj.SetEndLengthOffset("15",   # Label del elemento al que se le da brazo rígido, No valido (se aplica a todos al mismo tiempo)
                                            True,  # True para que se coloque en la distancia automáticamente
                                              0.5, # Distancia de brazo rígido inicial
                                              0.5, # Distancia de brazo rígido final
                                              0.5, # Rigidez utilizado
                                                2) # 0 = object, 1= Group, 2=SelectedObjects
#ret = SapModel.SelectObj.All(True)
ret

0

In [61]:
ret = SapModel.FrameObj.SetEndLengthOffset("ALL",   # Label del elemento al que se le da brazo rígido, No valido (se aplica a todos al mismo tiempo)
                                            True,  # True para que se coloque en la distancia automáticamente
                                              0.5, # Distancia de brazo rígido inicial
                                              0.5, # Distancia de brazo rígido final
                                              0.5, # Rigidez utilizado
                                                1) # 0 = object, 1= Group, 2=SelectedObjects

ret

0

## Asignación de diafragmas rígidos

### Definir y asignar diafragma rígido

In [63]:
# ret = SapModel.ConstraintDef.SetDiaphragm("Diaph1") # Ya no sirve, por defecto se crea D1

# Crear diafragma
ret = SapModel.Diaphragm.SetDiaphragm("MyDiaph1A", True)
#Asignar diafragma
#ret = SapModel.PointObj.SetDiaphragm("2", eDiaphragmOption.FromShellObject, "MyDiaph1A")
ret = SapModel.AreaObj.SetDiaphragm("1", "MyDiaph1A")

ret

0

In [64]:
#Asignar diafragma a un punto
ret = SapModel.PointObj.SetDiaphragm("2", 3, "D1")
ret = SapModel.PointObj.SetDiaphragm("3", 3, "D1")
ret = SapModel.PointObj.SetDiaphragm("6", 3, "D1")
ret = SapModel.PointObj.SetDiaphragm("7", 3, "D1")
ret

0

## Guardar, correr y sacar candado

### Guardar programa y colocar ubicación de archivo

De preferencia guardar en una carpeta del disco "C"
Porque en caso de guardarse en otra, al correr el modelo
van a salir ventanas emergentes que hay que darles "ok"
y no correrá de forma automática

In [65]:
# GUARDAR
# Nombre de la carpeta donde se guardará por defecto si no se especifica ninguna

APIPath = 'C:\PRUEBACURSO_PYTHON_ETABS'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'MOD1.edb'

ret = SapModel.File.Save(ModelPath)

### Correr software

In [66]:
ret = SapModel.Analyze.RunAnalysis()

### Quitar el candado

In [67]:
ret = SapModel.SetModelIsLocked(False)

### Salir del software

In [68]:
#close the program
ret = myETABSObject.ApplicationExit(False)
SapModel = None
myETABSObject = None

## Agregar cargas a pórticos y losas

### Agregar cargas puntuales en puntos

In [254]:
PointName1 = ' '
PointName2 = ' '

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
PointLoadValue = [0,0,-10,0,0,0]
ret = SapModel.PointObj.SetLoadForce(PointName2, 'DEAD', PointLoadValue)

### Agregar cargas distribuida a elementos tipo Frame

In [261]:
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2,   #Nombre del elemento
                                             'Dead',     #Nombre del tipo de carga
                                              1,         #1 fuerza/m y 2 es momento/m
                                              9,         #9 es gravedad, el de la pag no sirve
                                              0,         #Varia entre 0 y 1, distancia de inicio respecto del inicio
                                              1,         #Varia entre 0 y 1, distancia de fin respecto del inicio
                                              2,         #Carga inicial
                                              2,         #Carga final
                                              )

                                   

### Agregar cargas distrubuidas a elementos tipo AREA

In [285]:
ret = SapModel.AreaObj.SetLoadUniform("1",       # El nombre del elemento tipo área
                                     "DEAD",     # Tipo de carga a aplicar
                                      -5,         # Carga a aplicar 
                                      6,         # 6 Z al ser negativo la carga sup se vuelve gravedad
                                      False,
                                      "GLOBAL")